In [1]:
#This is code just for the intial J3 demo rollup
#In the future, I will need to include geometry validation code


#Import the libraries I need
import pandas as pd
import geopandas as gpd
import matplotlib as plt
from shapely import wkt
from shapely.geometry import Point

In [2]:
#Specify the geojson file path to read as a geodataframe
SMS = gpd.read_file(r"C:\Users\mjeas\Desktop\geojson (1).json")

In [3]:
#Pull out the point features from the main dataset
SMS_Points= SMS.loc[SMS['shapeType'] == 'Point']

In [4]:
#Pull out the line features from the main dataset
SMS_Lines= SMS.loc[SMS['shapeType'] == 'LINE']

In [5]:
#Pull out the polygons and circle features from the main dataset
SMS_Polygons= SMS.loc[SMS['shapeType'] == 'POLYGON']
SMS_Circle = SMS.loc[SMS['shapeType'] == 'CIRCLE']

In [6]:
#Data Cleaning
#slice the string to get rid of the brackets
SMS_Circle['center']= SMS_Circle['center'].str.slice(start=1, stop=-1)

#iterate over objects in 'center' column and add "POINT"
def point_format(x):
    return 'POINT' + '('+ x + ')'
SMS_Circle['center'] = SMS_Circle.apply(lambda row: point_format(row['center']), axis=1)

#Get rid of comma seperation
SMS_Circle['center'] = SMS_Circle['center'].replace(',',' ', regex=True)

C:\Users\mjeas\anaconda3\envs\geopandas_env\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [7]:
#Move the geometry format created in the last step to the geometry column and load it
SMS_Circle['geometry'] = SMS_Circle['center'].apply(wkt.loads)

In [8]:
#Set up inputs for buffer tool
sep = '.'
SMS_Circle['buff_distance'] = SMS_Circle['radius'].str.split(sep, 1, expand= True)[0]
SMS_Circle['buff_distance']= SMS_Circle['buff_distance'].astype(int)

In [9]:
#Reproject everything
SMS_Circle= SMS_Circle.to_crs("EPSG:3857")
SMS_Points= SMS_Points.to_crs("EPSG:3857")
SMS_Lines= SMS_Lines.to_crs("EPSG:3857")
SMS_Polygons= SMS_Polygons.to_crs("EPSG:3857")

#Perform buffer
SMS_Circle['geometry2'] = SMS_Circle.geometry.buffer(SMS_Circle['buff_distance'])
SMS_Circle = SMS_Circle.set_geometry('geometry2')

In [10]:
#delete columns
del SMS_Circle['geometry']
del SMS_Circle['buff_distance']
#del SMS_Circle['center']

In [12]:
#rename new geometry field
SMS_Circle.rename(columns= {'geometry2':'geometry'}, inplace=True)

#resetting the geometry field
SMS_Circle = SMS_Circle.set_geometry('geometry')

#Since cirlces are now polygons as they need to be, we can merge them back to the main polygon dataframe
SMS_Polygons= pd.concat([SMS_Polygons, SMS_Circle])

In [14]:
#convert to ESRI shapefiles
#I point thse to a geodatabae file that I had already created
#Future improvement: Add in driver for FileGDB. Requires some extra enviornmental work
SMS_Points.to_file('C:/Users/mjeas/Desktop/J3/SMS_Points.shp', driver='ESRI Shapefile')
SMS_Lines.to_file('C:/Users/mjeas/Desktop/J3/SMS_Lines.shp', driver='ESRI Shapefile')
SMS_Polygons.to_file('C:/Users/mjeas/Desktop/J3/SMS_Polygons.shp', driver='ESRI Shapefile')